In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetch your Kaggle API token and .json file. Place in the /content.

In [ ]:
path = '/content/drive/MyDrive/Kaggle/'

In [ ]:
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import pickle
import gc

In [ ]:
train = pd.read_csv(path+'train.csv')
test_unlabeled = pd.read_csv(path+'test_unlabeled.csv')


In [ ]:
filenames = !ls {path}chunk*.csv
filenames

['/content/drive/MyDrive/Kaggle/chunk_10.csv',
 '/content/drive/MyDrive/Kaggle/chunk_11.csv',
 '/content/drive/MyDrive/Kaggle/chunk_12.csv',
 '/content/drive/MyDrive/Kaggle/chunk_13.csv',
 '/content/drive/MyDrive/Kaggle/chunk_14.csv',
 '/content/drive/MyDrive/Kaggle/chunk_15.csv',
 '/content/drive/MyDrive/Kaggle/chunk_16.csv',
 '/content/drive/MyDrive/Kaggle/chunk_17.csv',
 '/content/drive/MyDrive/Kaggle/chunk_18.csv',
 '/content/drive/MyDrive/Kaggle/chunk_19.csv',
 '/content/drive/MyDrive/Kaggle/chunk_1.csv',
 '/content/drive/MyDrive/Kaggle/chunk_20.csv',
 '/content/drive/MyDrive/Kaggle/chunk_21.csv',
 '/content/drive/MyDrive/Kaggle/chunk_22.csv',
 '/content/drive/MyDrive/Kaggle/chunk_23.csv',
 '/content/drive/MyDrive/Kaggle/chunk_24.csv',
 '/content/drive/MyDrive/Kaggle/chunk_25.csv',
 '/content/drive/MyDrive/Kaggle/chunk_26.csv',
 '/content/drive/MyDrive/Kaggle/chunk_27.csv',
 '/content/drive/MyDrive/Kaggle/chunk_28.csv',
 '/content/drive/MyDrive/Kaggle/chunk_29.csv',
 '/content/dri

In [ ]:
train.head()

,Unnamed: 0,mint,slot_min,slot_graduated,has_graduated,is_valid
0,1,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,317876496,NaN,False,True
1,3,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,317876500,NaN,False,True
2,4,DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump,317876500,NaN,False,True
3,5,6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump,317876500,NaN,False,True
4,6,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,317876500,NaN,False,True


In [ ]:
df_sample_chunk_head = pd.read_csv(filenames[1], nrows=5)
# print(df_sample_chunk_head)
print(df_sample_chunk_head.columns.tolist())

['block_time', 'slot', 'tx_idx', 'signing_wallet', 'direction', 'base_coin', 'base_coin_amount', 'quote_coin_amount', 'virtual_token_balance_after', 'virtual_sol_balance_after', 'signature', 'provided_gas_fee', 'provided_gas_limit', 'fee', 'consumed_gas']


In [ ]:
df_sample_chunk_head = pd.read_csv(path+'token_info_onchain_divers_v2.csv', nrows=5)
# print(df_sample_chunk_head)
print(df_sample_chunk_head.columns.tolist())

['block_time', 'slot', 'tx_idx', 'creator', 'name', 'symbol', 'url', 'mint', 'bundle_size', 'gas_used', 'amount_of_instructions', 'amount_of_lookup_reads', 'amount_of_lookup_writes', 'bundle_structure', 'bundled_buys', 'bundled_buys_count', 'dev_balance', 'creation_ix_index', 'curve_address', 'pf_program_index', 'direct_pf_invocation', 'version']


In [ ]:
MAX_SLOT_OFFSET = 100
metadata_mint_col = 'mint'
metadata_creation_slot_col = 'slot'
train_test_mint_col = 'mint'

metadata_filename = 'token_info_onchain_divers_v2.csv'
train_filename = 'train.csv'
test_filename = 'test_unlabeled.csv'

In [ ]:
train_df = pd.read_csv(path+train_filename, usecols=[train_test_mint_col])
test_df = pd.read_csv(path+test_filename, usecols=[train_test_mint_col])
metadata_df = pd.read_csv(path+metadata_filename, usecols=[metadata_mint_col, metadata_creation_slot_col])

Combine and Dedupe the Mints

In [ ]:
all_mints = pd.concat([train_df[[train_test_mint_col]], test_df[[train_test_mint_col]]], ignore_index=True)
unique_mints_df = all_mints.drop_duplicates().reset_index(drop=True)
print(f"Total unique mints from train and test sets: {len(unique_mints_df)}")

Total unique mints from train and test sets: 1118389


Merge creation slots

In [ ]:
metadata_df = metadata_df.drop_duplicates(subset=metadata_mint_col)

# Merge metadata (creation slot) onto the unique mint list
# Use left merge to keep all unique mints, even if metadata is missing for some
mints_info_target = pd.merge(
    unique_mints_df,
    metadata_df,
    left_on=train_test_mint_col, # 'mint' from unique_mints_df
    right_on=metadata_mint_col, # 'mint' from metadata_df
    how='left'
)

In [ ]:
print(f"Shape after merging metadata: {mints_info_target.shape}")

missing_slots = mints_info_target[metadata_creation_slot_col].isnull().sum()
print(f"\nNumber of mints missing creation slot ('{metadata_creation_slot_col}') info: {missing_slots}")

Shape after merging metadata: (1118389, 2)

Number of mints missing creation slot ('slot') info: 55165


Dropping missing creation splots

In [ ]:
mints_info_target = mints_info_target.dropna(subset=[metadata_creation_slot_col]).copy()
print(f"Proceeding with {len(mints_info_target)} mints that have creation slot information.")

Proceeding with 1063224 mints that have creation slot information.


Calculate the slot ranges

In [ ]:
# Ensure creation slot is integer type (originally was int64)
mints_info_target[metadata_creation_slot_col] = mints_info_target[metadata_creation_slot_col].astype(int)

In [ ]:
# Define the start and end slot for the feature window
mints_info_target['target_slot_start'] = mints_info_target[metadata_creation_slot_col]
mints_info_target['target_slot_end'] = mints_info_target[metadata_creation_slot_col] + MAX_SLOT_OFFSET

In [ ]:
# Clean up columns - keep only necessary info for processing chunks
mints_info_target = mints_info_target[[
    train_test_mint_col, # The actual mint address
    'target_slot_start',
    'target_slot_end'
]].rename(columns={train_test_mint_col: 'mint'}) # Standardize mint column name to 'mint'

In [ ]:
print(f"Shape of final DataFrame: {mints_info_target.shape}")
print(mints_info_target.head())
print(mints_info_target.info())

Shape of final DataFrame: (1063224, 3)
                                           mint  target_slot_start  \
0  BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump          317876496   
1  4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump          317876500   
2  DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump          317876500   
3  6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump          317876500   
4  4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump          317876500   

   target_slot_end  
0        317876596  
1        317876600  
2        317876600  
3        317876600  
4        317876600  
<class 'pandas.core.frame.DataFrame'>
Index: 1063224 entries, 0 to 1118388
Data columns (total 3 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   mint               1063224 non-null  object
 1   target_slot_start  1063224 non-null  int64 
 2   target_slot_end    1063224 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 32.4+ MB
None


In [ ]:
mints_info_target_save_path = os.path.join(path, 'mints_info_target_prepared.parquet')

try:
    mints_info_target.to_parquet(mints_info_target_save_path) # Keep index by default
    print(f"Successfully saved mints_info_target to: {mints_info_target_save_path}")
except Exception as e:
    print(f"ERROR saving mints_info_target: {e}")
# --- End Save ---

Successfully saved mints_info_target to: /content/drive/MyDrive/Kaggle/mints_info_target_prepared.parquet


# Read for aggregation and feature engineering

We are transforming the raw, time-series transaction data from the chunks into a fixed set of summary statistics (features) for each mint, based only on its first 100 slots of activity.

In [ ]:
chunk_files = sorted(glob.glob(os.path.join(path, 'chunk_*.csv')))
chunk_files

['/content/drive/MyDrive/Kaggle/chunk_1.csv',
 '/content/drive/MyDrive/Kaggle/chunk_10.csv',
 '/content/drive/MyDrive/Kaggle/chunk_11.csv',
 '/content/drive/MyDrive/Kaggle/chunk_12.csv',
 '/content/drive/MyDrive/Kaggle/chunk_13.csv',
 '/content/drive/MyDrive/Kaggle/chunk_14.csv',
 '/content/drive/MyDrive/Kaggle/chunk_15.csv',
 '/content/drive/MyDrive/Kaggle/chunk_16.csv',
 '/content/drive/MyDrive/Kaggle/chunk_17.csv',
 '/content/drive/MyDrive/Kaggle/chunk_18.csv',
 '/content/drive/MyDrive/Kaggle/chunk_19.csv',
 '/content/drive/MyDrive/Kaggle/chunk_2.csv',
 '/content/drive/MyDrive/Kaggle/chunk_20.csv',
 '/content/drive/MyDrive/Kaggle/chunk_21.csv',
 '/content/drive/MyDrive/Kaggle/chunk_22.csv',
 '/content/drive/MyDrive/Kaggle/chunk_23.csv',
 '/content/drive/MyDrive/Kaggle/chunk_24.csv',
 '/content/drive/MyDrive/Kaggle/chunk_25.csv',
 '/content/drive/MyDrive/Kaggle/chunk_26.csv',
 '/content/drive/MyDrive/Kaggle/chunk_27.csv',
 '/content/drive/MyDrive/Kaggle/chunk_28.csv',
 '/content/driv

In [ ]:
df_sample_chunk_head = pd.read_csv(filenames[1], nrows=5)
print(df_sample_chunk_head)

            block_time       slot  tx_idx  \
0  2025-02-07 19:37:38  319209603      29   
1  2025-02-07 19:37:38  319209603     389   
2  2025-02-07 19:37:38  319209603      92   
3  2025-02-07 19:37:38  319209603     154   
4  2025-02-07 19:37:38  319209603    1260   

                                 signing_wallet direction  \
0  Ea8wdeVtJKvDxBFditrWsK2mfibnLRAea5MN2zDHAyp5       buy   
1  5YET3YapxD6to6rqPqTWB3R9pSbURy6yduuUtoZkzoPX       buy   
2  E4Q4x2R59yrA39uSdfBW7GycCiP67yhVWsybdZtqMLQa       buy   
3  ATtBj32ooZb3PToqTjV8rBW3XtUEp8xXWmv6LZR3PrRZ       buy   
4   o7RY6P2vQMuGSu1TrLM81weuzgDjaCRTXYRaXJwWcvc       buy   

                                      base_coin  base_coin_amount  \
0  8u2XH268edziKqBRNXKReVBnqMupiRL3w7RWAU8rpump      164980128238   
1  8u2XH268edziKqBRNXKReVBnqMupiRL3w7RWAU8rpump      341357517312   
2  BF6wPpSPpWEbEq3wcyfnmwDjYXGa443DhvkyNiitpump      352511236077   
3  H6mFjATx2yx8NQtYuzf6KfYuu3crDLTtn2M5Y5G8pump    34612903225806   
4  H6mFjATx2yx8NQ

In [ ]:
COLS_TO_LOAD_FROM_CHUNKS = [
    'block_time', # Timestamp (useful for time-based features)
    'slot', # The block/slot number
    'tx_idx',
    'signing_wallet', # Wallet initiating the swap
    'direction',  # 'buy' or 'sell'
    'base_coin', # The mint address
    'base_coin_amount',
    'quote_coin_amount', # SOL amount involved
    'virtual_token_balance_after', # State of the virtual LP
    'virtual_sol_balance_after',
    'signature',
    'provided_gas_fee',
    'provided_gas_limit',
    'fee',
    'consumed_gas'
  ]

**Phase 1: Analyzing Column Potential (Step-by-Step Thinking)**

We'll examine each column from `COLS_TO_LOAD_FROM_CHUNKS` and brainstorm its potential value within the "first 100 slots" constraint:

1.  **`block_time` (Timestamp):**
    *   **Direct Use:** Can calculate the precise duration (in seconds) of activity within the 100-slot window (`last_time - first_time`).
    *   **Engineered Features:**
        *   `time_span_seconds_100s`: The real-world time duration of observed activity. Short spans might indicate bursts.
        *   `tx_per_second_100s`: Transaction velocity (`tx_count / time_span_seconds`). High values suggest intense early interest.
        *   `sol_volume_per_second_100s`: Economic velocity.
        *   Time-weighted features (advanced): Give more weight to earlier or later transactions within the window.
    *   **Potential Signal:** Measures real-world speed of interaction, complementing the block-based `slot` timing. Captures potential bursts of activity that might occur within a single slot.

2.  **`slot` (Block/Slot Number):**
    *   **Direct Use:** Essential for filtering transactions to the `[creation_slot, creation_slot + 100)` window.
    *   **Engineered Features:**
        *   `first_slot_offset_100s`: `first_observed_slot - creation_slot`. How quickly did activity *start* after creation? (Value near 0 is expected if active immediately).
        *   `last_slot_offset_100s`: `last_observed_slot - creation_slot`. How long did activity persist within the window? (Value near 99 suggests sustained activity).
        *   `slot_span_100s`: `last_observed_slot - first_observed_slot`. How many slots saw activity?
        *   `tx_per_slot_100s`: Density of transactions across active slots (`tx_count / (slot_span + 1)`).
        *   `activity_slot_density_100s`: `(slot_span + 1) / 100`. What fraction of the window saw *any* activity?
    *   **Potential Signal:** Measures activity timing and duration relative to blockchain blocks.

3.  **`tx_idx` (Transaction Index within Slot):**
    *   **Direct Use:** Indicates order within a slot.
    *   **Engineered Features:**
        *   `avg_tx_idx_100s`: Average position within a block. Consistently high values might indicate getting included later in blocks.
        *   `max_tx_idx_100s`: Maximum index seen. Could suggest high contention slots.
        *   `stddev_tx_idx_100s`: Variability in transaction position.
    *   **Potential Signal:** Subtle signal, might reflect transaction priority, network congestion during interaction, or bot behavior landing transactions at specific points in blocks. Likely less impactful than volume/wallet features.

4.  **`signing_wallet` (Wallet Initiating Swap):**
    *   **Direct Use:** Identifier for the actor.
    *   **Engineered Features:**
        *   `unique_wallets_count_100s`: Total distinct participants. Key indicator of community size/interest.
        *   `unique_buyers_count_100s`: Distinct wallets performing 'buy'.
        *   `unique_sellers_count_100s`: Distinct wallets performing 'sell'.
        *   `new_wallet_rate_100s`: `unique_wallets_count / slot_span` or `/ time_span_seconds`. How quickly are new participants joining?
        *   `avg_tx_per_wallet_100s`: `tx_count / unique_wallets_count`. High value might indicate bots or very active individuals.
        *   `avg_sol_vol_per_wallet_100s`: `total_sol_volume / unique_wallets_count`.
        *   `concentration_metrics (Advanced)`: E.g., % of volume from top 1/3/5 wallets. Requires storing per-wallet data, memory-intensive. Start without, add later if needed.
        *   `creator_activity (Advanced)`: Requires linking `signing_wallet` to the `creator` from metadata. Did the creator buy/sell early? Complex join.
    *   **Potential Signal:** Extremely important. Reflects breadth and depth of engagement, potential bot activity, and overall community interest.

5.  **`direction` ('buy' or 'sell'):**
    *   **Direct Use:** Categorical flag for transaction type.
    *   **Engineered Features:** (Many features depend on this)
        *   `buy_tx_count_100s`, `sell_tx_count_100s`.
        *   `buy_sell_tx_ratio_100s`: `buy_count / sell_count`. High ratio indicates buying pressure.
        *   Dominant direction features: Was the first/last tx a buy or sell? Percentage of buys.
    *   **Potential Signal:** Fundamental indicator of market sentiment within the crucial early window.

6.  **`base_coin` (Token Mint Address):**
    *   **Direct Use:** The primary key for grouping transactions per token. Not a feature itself.

7.  **`base_coin_amount` (Token Amount):**
    *   **Direct Use:** Quantity of the memecoin traded.
    *   **Engineered Features:**
        *   `buy_token_vol_100s`, `sell_token_vol_100s`, `total_token_vol_100s`, `net_token_vol_100s`.
        *   `avg_buy_token_100s`, `avg_sell_token_100s`, `avg_tx_token_100s`.
        *   `stddev_token_vol_100s` (Advanced, if memory allows).
        *   Used in implied price calculations.
    *   **Potential Signal:** Reflects activity in terms of the token itself. Large average sizes might indicate whales. Complements SOL volume.

8.  **`quote_coin_amount` (SOL Amount):**
    *   **Direct Use:** SOL value exchanged. Primary economic indicator.
    *   **Engineered Features:**
        *   `buy_sol_vol_100s`, `sell_sol_vol_100s`, `total_sol_vol_100s`, `net_sol_vol_100s` (BUY - SELL, critical measure of net inflow).
        *   `avg_buy_sol_100s`, `avg_sell_sol_100s`, `avg_tx_sol_100s`.
        *   `stddev_sol_vol_100s` (Advanced, if memory allows).
        *   `max_buy_sol_tx_100s`, `max_sell_sol_tx_100s`: Size of largest individual buy/sell.
    *   **Potential Signal:** Highly important. Directly measures the SOL flowing into/out of the token, reflecting demand and perceived value. Net volume is key.

9.  **`virtual_token_balance_after` (Token side of LP):**
    *   **Direct Use:** State of the token pool after the transaction.
    *   **Engineered Features:**
        *   `lp_token_start_100s`, `lp_token_end_100s`.
        *   `lp_token_change_100s`: `end - start`.
        *   `lp_token_change_pct_100s`: `(end - start) / start`.
        *   `lp_token_min_100s`, `lp_token_max_100s`: Range of LP token balance during the window.
        *   Used in implied price calculations.
    *   **Potential Signal:** Tracks the supply dynamics within the bonding curve's virtual pool.

10. **`virtual_sol_balance_after` (SOL side of LP):**
    *   **Direct Use:** State of the SOL pool after the transaction. Crucial liquidity indicator.
    *   **Engineered Features:**
        *   `lp_sol_start_100s`, `lp_sol_end_100s`. (End value is a strong candidate feature itself).
        *   `lp_sol_change_100s`: `end - start`. Key indicator of net liquidity change.
        *   `lp_sol_change_pct_100s`: `(end - start) / start`. Relative growth.
        *   `lp_sol_min_100s`, `lp_sol_max_100s`: Range of SOL liquidity during the window.
        *   `rate_lp_sol_change_100s`: `lp_sol_change / slot_span` or `/ time_span_seconds`. How fast is liquidity accumulating?
    *   **Potential Signal:** Extremely important. Directly tracks the accumulation of SOL liquidity, which is the basis for graduation. The ending value and the change are critical.

11. **`signature` (Transaction Signature):**
    *   **Direct Use:** Unique identifier.
    *   **Engineered Features:** Could potentially count unique signatures if `tx_count` is suspected to double-count retry attempts, but unlikely necessary.
    *   **Potential Signal:** Low direct value for prediction.

12. **`provided_gas_fee`, `provided_gas_limit`, `fee`, `consumed_gas`:**
    *   **Direct Use:** Details about transaction costs.
    *   **Engineered Features:**
        *   `total_gas_consumed_100s`, `avg_gas_consumed_100s`, `stddev_gas_consumed_100s`.
        *   `total_fee_100s`, `avg_fee_100s`, `stddev_fee_100s`.
        *   `avg_gas_efficiency_100s`: `avg(consumed / provided_limit)`.
        *   `max_fee_100s`, `min_fee_100s`.
    *   **Potential Signal:** Might reflect user sophistication, urgency (high fees for priority), or bot patterns (consistent fees/gas usage). Potentially subtle signals.


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm.notebook import tqdm
import time
import warnings

In [ ]:
# Suppress division by zero warnings during calculation, handle with np.inf/nan
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
path = '/content/drive/MyDrive/Kaggle/'
chunk_files = sorted(glob.glob(os.path.join(path, 'chunk_*.csv')))

In [ ]:
COLS_TO_LOAD_FROM_CHUNKS = [
    'block_time', 'slot', 'tx_idx', 'signing_wallet', 'direction', 'base_coin',
    'base_coin_amount', 'quote_coin_amount', 'virtual_token_balance_after',
    'virtual_sol_balance_after', 'signature', 'provided_gas_fee',
    'provided_gas_limit', 'fee', 'consumed_gas'
]

In [ ]:
# --- Step 1: Prepare Mint Lookup ---
target_mints_dict = mints_info_target.set_index('mint').to_dict('index')
print(f"Lookup dictionary created for {len(target_mints_dict)} mints.")

NameError: name 'mints_info_target' is not defined

In [ ]:
# --- Step 2: Initialize Feature Storage ---
# This dictionary structure is defined *inside* the loop when a mint is first seen.
# The keys listed here are just for reference of what will be created.
_initial_feature_structure_template = {
    'mint': '', # Set to the mint address
    'tx_count_100s': 0,
    'buy_tx_count_100s': 0, 'sell_tx_count_100s': 0,
    'total_sol_vol_100s': 0.0, 'buy_sol_vol_100s': 0.0, 'sell_sol_vol_100s': 0.0,
    'total_token_vol_100s': 0.0, 'buy_token_vol_100s': 0.0, 'sell_token_vol_100s': 0.0,
    'total_gas_consumed_100s': 0.0, 'total_fee_100s': 0.0,
    'unique_wallets_set': set(), 'unique_buyers_set': set(), 'unique_sellers_set': set(),
    'first_slot_100s': -1, 'last_slot_100s': -1, # Placeholder, set on first tx
    'first_time_100s': pd.NaT, 'last_time_100s': pd.NaT, # Placeholder, set on first tx
    'lp_sol_start_100s': 0.0, 'lp_sol_end_100s': 0.0,
    'lp_token_start_100s': 0.0, 'lp_token_end_100s': 0.0,
    'min_lp_sol_100s': np.inf, 'max_lp_sol_100s': -np.inf, # Initialize correctly for min/max
    'min_lp_token_100s': np.inf, 'max_lp_token_100s': -np.inf,
    'sum_tx_idx_100s': 0, 'max_tx_idx_100s': -1,
    'max_buy_sol_tx_100s': 0.0, 'max_sell_sol_tx_100s': 0.0,

    # --- NEW V2 Features Initialization ---
    # Half-window accumulators
    'tx_count_first_half': 0,
    'sol_vol_first_half': 0.0,
    'wallets_first_half_set': set(),
    'lp_sol_at_half_way_mark': 0.0, # LP SOL balance after last tx in first half
    'tx_count_second_half': 0,
    'sol_vol_second_half': 0.0,
    'wallets_second_half_set': set(), # Will contain wallets from *both* halves, need to diff later

    # Gas limit accumulator
    'total_provided_gas_limit_100s': 0.0,

    # Placeholder for volatility lists (if implemented later)
    # 'tx_sol_amounts_list': [],
    # 'lp_sol_values_list': [],
}

features_per_mint = {} # This will store the actual data, keyed by mint

In [ ]:
# --- Step 3: Process Chunks Incrementally ---
print(f"Starting chunk processing...")
start_time = time.time()
total_relevant_rows = 0
processed_chunks_count = 0

# Clear previous data if re-running
features_per_mint = {}

for i, chunk_filename in enumerate(tqdm(chunk_files, desc="Processing Chunks")):
    try:
        df_chunk = pd.read_csv(
            chunk_filename,
            usecols=COLS_TO_LOAD_FROM_CHUNKS,
            dtype={
                'slot': 'int64', 'tx_idx': 'int16', 'direction': 'category',
                'base_coin': 'object', 'signing_wallet': 'object',
                'base_coin_amount': 'float64', 'quote_coin_amount': 'float64',
                'virtual_token_balance_after': 'float64', 'virtual_sol_balance_after': 'float64',
                'signature': 'object', 'provided_gas_fee': 'float64',
                'provided_gas_limit': 'float64', # Ensure this is loaded
                'fee': 'float64', 'consumed_gas': 'float64'
            },
            parse_dates=['block_time'],
            infer_datetime_format=True,
            low_memory=False
        )

        # Pre-filter chunk for potentially relevant mints (optimization)
        chunk_mints_set = set(df_chunk['base_coin'].unique())
        relevant_mints_in_chunk = chunk_mints_set.intersection(target_mints_dict.keys())
        if not relevant_mints_in_chunk:
             processed_chunks_count += 1
             continue # Skip chunk if no relevant mints

        df_chunk_filtered = df_chunk[df_chunk['base_coin'].isin(relevant_mints_in_chunk)].copy()
        # Sort chunk by slot then tx_idx to process transactions in order (important for LP state)
        df_chunk_filtered.sort_values(by=['slot', 'tx_idx'], inplace=True)


        # Iterate through relevant rows efficiently
        for row in df_chunk_filtered.itertuples(index=False):
            mint = row.base_coin
            slot = row.slot

            # Check if mint is one we are tracking (already pre-filtered, but double check)
            if mint in target_mints_dict:
                mint_info = target_mints_dict[mint]
                target_start = mint_info['target_slot_start']
                target_end = mint_info['target_slot_end']
                midpoint_slot = target_start + 50 # Define the halfway point

                # Check if the transaction falls within the 100-slot window
                if target_start <= slot < target_end:
                    total_relevant_rows += 1

                    # Extract data from row, handle potential NaNs
                    direction = row.direction
                    wallet = row.signing_wallet
                    sol_amount = row.quote_coin_amount if pd.notna(row.quote_coin_amount) else 0.0
                    token_amount = row.base_coin_amount if pd.notna(row.base_coin_amount) else 0.0
                    consumed_gas = row.consumed_gas if pd.notna(row.consumed_gas) else 0.0
                    fee = row.fee if pd.notna(row.fee) else 0.0
                    tx_idx = row.tx_idx if pd.notna(row.tx_idx) else 0
                    provided_limit = row.provided_gas_limit if pd.notna(row.provided_gas_limit) else 0.0
                    current_block_time = row.block_time if pd.notna(row.block_time) else pd.NaT
                    current_lp_sol = row.virtual_sol_balance_after if pd.notna(row.virtual_sol_balance_after) else 0.0
                    current_lp_token = row.virtual_token_balance_after if pd.notna(row.virtual_token_balance_after) else 0.0


                    # Initialize storage for this mint if first time seeing it within the window
                    if mint not in features_per_mint:
                        features_per_mint[mint] = {
                            'mint': mint,
                            'tx_count_100s': 0,
                            'buy_tx_count_100s': 0, 'sell_tx_count_100s': 0,
                            'total_sol_vol_100s': 0.0, 'buy_sol_vol_100s': 0.0, 'sell_sol_vol_100s': 0.0,
                            'total_token_vol_100s': 0.0, 'buy_token_vol_100s': 0.0, 'sell_token_vol_100s': 0.0,
                            'total_gas_consumed_100s': 0.0, 'total_fee_100s': 0.0,
                            'unique_wallets_set': set(), 'unique_buyers_set': set(), 'unique_sellers_set': set(),
                            'first_slot_100s': slot, 'last_slot_100s': slot, # Initial values
                            'first_time_100s': current_block_time, 'last_time_100s': current_block_time,
                            'lp_sol_start_100s': current_lp_sol, 'lp_sol_end_100s': current_lp_sol, # Initial state from first tx
                            'lp_token_start_100s': current_lp_token, 'lp_token_end_100s': current_lp_token,
                            'min_lp_sol_100s': current_lp_sol, 'max_lp_sol_100s': current_lp_sol,
                            'min_lp_token_100s': current_lp_token, 'max_lp_token_100s': current_lp_token,
                            'sum_tx_idx_100s': 0, 'max_tx_idx_100s': tx_idx,
                            'max_buy_sol_tx_100s': 0.0, 'max_sell_sol_tx_100s': 0.0,
                            # V2 Init
                            'tx_count_first_half': 0, 'sol_vol_first_half': 0.0,
                            'wallets_first_half_set': set(),
                            'lp_sol_at_half_way_mark': current_lp_sol, # Initialize with start value, update below
                            'tx_count_second_half': 0, 'sol_vol_second_half': 0.0,
                            'wallets_second_half_set': set(),
                            'total_provided_gas_limit_100s': 0.0,
                        }

                    # Get reference to the mint's feature dictionary
                    state = features_per_mint[mint]

                    # --- Update Overall State (Accumulate/Track) ---
                    state['tx_count_100s'] += 1
                    state['unique_wallets_set'].add(wallet)
                    state['total_sol_vol_100s'] += sol_amount
                    state['total_token_vol_100s'] += token_amount
                    state['total_gas_consumed_100s'] += consumed_gas
                    state['total_fee_100s'] += fee
                    state['sum_tx_idx_100s'] += tx_idx
                    state['max_tx_idx_100s'] = max(state['max_tx_idx_100s'], tx_idx)
                    state['total_provided_gas_limit_100s'] += provided_limit # V2 Gas Limit

                    # Track min/max/last slots and times (update only if not NaT)
                    state['last_slot_100s'] = max(state['last_slot_100s'], slot)
                    if pd.notna(current_block_time):
                        state['last_time_100s'] = max(state['last_time_100s'], current_block_time)
                        # Update first time if it was NaT initially and this is the first valid time
                        if pd.isna(state['first_time_100s']):
                             state['first_time_100s'] = current_block_time

                    # Track min/max/end LP values (use the current values calculated earlier)
                    state['lp_sol_end_100s'] = current_lp_sol
                    state['lp_token_end_100s'] = current_lp_token
                    state['min_lp_sol_100s'] = min(state['min_lp_sol_100s'], current_lp_sol)
                    state['max_lp_sol_100s'] = max(state['max_lp_sol_100s'], current_lp_sol)
                    state['min_lp_token_100s'] = min(state['min_lp_token_100s'], current_lp_token)
                    state['max_lp_token_100s'] = max(state['max_lp_token_100s'], current_lp_token)

                    # --- V2: Update Half-Window Features ---
                    state['wallets_second_half_set'].add(wallet) # Add to second half set regardless (will diff later)
                    if slot < midpoint_slot:
                        # Update first half accumulators
                        state['tx_count_first_half'] += 1
                        state['sol_vol_first_half'] += sol_amount
                        state['wallets_first_half_set'].add(wallet)
                        # Keep track of the LP SOL balance after the last transaction in the first half
                        state['lp_sol_at_half_way_mark'] = current_lp_sol
                    else:
                        # Update second half accumulators
                        state['tx_count_second_half'] += 1
                        state['sol_vol_second_half'] += sol_amount
                        # Wallet added to wallets_second_half_set already

                    # --- Direction-specific updates ---
                    if direction == 'buy':
                        state['buy_tx_count_100s'] += 1
                        state['buy_sol_vol_100s'] += sol_amount
                        state['buy_token_vol_100s'] += token_amount
                        state['unique_buyers_set'].add(wallet)
                        state['max_buy_sol_tx_100s'] = max(state['max_buy_sol_tx_100s'], sol_amount)
                    elif direction == 'sell':
                        state['sell_tx_count_100s'] += 1
                        state['sell_sol_vol_100s'] += sol_amount
                        state['sell_token_vol_100s'] += token_amount
                        state['unique_sellers_set'].add(wallet)
                        state['max_sell_sol_tx_100s'] = max(state['max_sell_sol_tx_100s'], sol_amount)

        processed_chunks_count += 1
        # Optional: Intermediate saving/logging
        # if processed_chunks_count % 5 == 0: print(f"  Processed {processed_chunks_count}/{len(chunk_files)} chunks...")

    except FileNotFoundError:
        print(f"Warning: Chunk file not found: {chunk_filename}")
        continue
    except pd.errors.EmptyDataError:
         print(f"Warning: Chunk file is empty: {chunk_filename}")
         processed_chunks_count += 1
         continue
    except Exception as e:
        print(f"ERROR processing chunk {i} ({chunk_filename}): {type(e).__name__} - {e}")
        # Optional: Add more robust error handling / logging here
        continue # Continue to next chunk

end_time = time.time()
print(f"\nChunk processing finished. Time taken: {end_time - start_time:.2f} seconds.")
print(f"Total relevant transaction rows processed: {total_relevant_rows}")
print(f"Number of mints with features found: {len(features_per_mint)}")

# --- TECHNIQUE 3a: Save Intermediate Data ---
intermediate_save_path = os.path.join(path, 'intermediate_features_per_mint.pkl')
print(f"\nSaving intermediate features_per_mint to {intermediate_save_path}...")
try:
    with open(intermediate_save_path, 'wb') as f:
        pickle.dump(features_per_mint, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("Intermediate data saved successfully.")

    # Now delete the large object from memory
    print("Deleting features_per_mint from memory...")
    del features_per_mint # <<< Delete the main dictionary

    # Delete other potentially large loop variables
    print("Deleting loop variables...")
    try:
        del df_chunk
        del df_chunk_filtered
        del row
        del chunk_mints_set
        del relevant_mints_in_chunk
    except NameError:
        pass # Ignore if they don't exist

    # Force garbage collection
    print("Running garbage collection...")
    gc.collect()
    print("Garbage collection complete. Memory should be lower now.")

except Exception as e:
    print(f"ERROR saving or deleting intermediate data: {e}")
    # Decide if you want to proceed without reloading or stop
    raise e # Or handle differently
# --- End of Technique 3a ---

Starting chunk processing...


Processing Chunks:   0%|          | 0/41 [00:00<?, ?it/s]

<ipython-input-30-dbf59882a903>:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-30-dbf59882a903>:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-30-dbf59882a903>:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-30-d


Chunk processing finished. Time taken: 673.03 seconds.
Total relevant transaction rows processed: 16554095
Number of mints with features found: 1061444

Saving intermediate features_per_mint to /content/drive/MyDrive/Kaggle/intermediate_features_per_mint.pkl...
Intermediate data saved successfully.
Deleting features_per_mint from memory...
Deleting loop variables...
Running garbage collection...
ERROR saving or deleting intermediate data: name 'gc' is not defined


NameError: name 'gc' is not defined

In [ ]:
import gc
gc.collect()

700

In [ ]:
# --- Reload mints_info_target (Optional) ---
# If you have already run the preprocessing steps up to saving
# 'mints_info_target_prepared.parquet' and your kernel restarted,
# you can uncomment and run this cell to load it directly,
# instead of re-running all the cells from 'lXFLjCnEp4ep' onwards.

print("\nAttempting to reload pre-prepared mints_info_target...")
mints_info_target_load_path = os.path.join(path, 'mints_info_target_prepared.parquet')

try:
    if os.path.exists(mints_info_target_load_path):
        mints_info_target = pd.read_parquet(mints_info_target_load_path)
        print(f"Successfully reloaded mints_info_target from: {mints_info_target_load_path}")
        print(f"Reloaded DataFrame shape: {mints_info_target.shape}")
        # Optional: print head/info again to confirm
        # print(mints_info_target.head())
        # print(mints_info_target.info())
    else:
        print(f"ERROR: File not found at {mints_info_target_load_path}. You need to run the previous preprocessing cells first.")
        # Optionally raise an error to stop execution if the file is expected
        # raise FileNotFoundError(f"Required file not found: {mints_info_target_load_path}")
except Exception as e:
    print(f"ERROR reloading mints_info_target: {e}")
    # Raise error as proceeding might not work
    raise e
# --- End Reload ---


In [ ]:
# --- Step 4 (Modified Start): Reload Data ---

print("\nReloading intermediate features_per_mint...")
intermediate_save_path = os.path.join(path, 'intermediate_features_per_mint.pkl')

try:
    with open(intermediate_save_path, 'rb') as f:
        features_per_mint = pickle.load(f) # <<< Reload the dictionary
    print(f"Successfully reloaded {len(features_per_mint)} mints.")

    # Optional: Delete the pickle file now if you don't need it anymore
    # print(f"Deleting intermediate file {intermediate_save_path}...")
    # os.remove(intermediate_save_path)

except Exception as e:
    print(f"ERROR reloading intermediate data: {e}")
    # Cannot proceed without the data
    raise e
# --- End of Reload ---

# --- Step 4: Finalize Features and Create DataFrame ---
print("Finalizing features...")
final_features_list = []

# Create lookup map for creation slots more efficiently
creation_slots_map = mints_info_target.set_index('mint')['target_slot_start'].to_dict()

# Define a helper function for safe division
def safe_divide(numerator, denominator, default=np.nan):
    if denominator is None or denominator == 0 or pd.isna(denominator):
        return default
    try:
        result = numerator / denominator
        # Replace potential inf/-inf resulting from very small denominators too
        return result if np.isfinite(result) else default
    except (ZeroDivisionError, TypeError, ValueError):
        return default

for mint, features in tqdm(features_per_mint.items(), desc="Finalizing Features"):
    # Use 'state' alias for clarity
    state = features
    tx_count = state['tx_count_100s']
    if tx_count == 0: continue # Skip mints with no transactions found in window

    # --- Basic Features (Counts, Averages, Net, Ratios) ---
    state['unique_wallets_count_100s'] = len(state['unique_wallets_set'])
    state['unique_buyers_count_100s'] = len(state['unique_buyers_set'])
    state['unique_sellers_count_100s'] = len(state['unique_sellers_set'])

    state['avg_tx_idx_100s'] = safe_divide(state['sum_tx_idx_100s'], tx_count, default=0.0)
    state['avg_gas_consumed_100s'] = safe_divide(state['total_gas_consumed_100s'], tx_count, default=0.0)
    state['avg_fee_100s'] = safe_divide(state['total_fee_100s'], tx_count, default=0.0)
    state['avg_sol_vol_per_tx_100s'] = safe_divide(state['total_sol_vol_100s'], tx_count, default=0.0)
    state['avg_token_vol_per_tx_100s'] = safe_divide(state['total_token_vol_100s'], tx_count, default=0.0)
    state['avg_buy_sol_100s'] = safe_divide(state['buy_sol_vol_100s'], state['buy_tx_count_100s'], default=0.0)
    state['avg_sell_sol_100s'] = safe_divide(state['sell_sol_vol_100s'], state['sell_tx_count_100s'], default=0.0)
    state['avg_buy_token_100s'] = safe_divide(state['buy_token_vol_100s'], state['buy_tx_count_100s'], default=0.0)
    state['avg_sell_token_100s'] = safe_divide(state['sell_token_vol_100s'], state['sell_tx_count_100s'], default=0.0)
    state['avg_sol_vol_per_wallet_100s'] = safe_divide(state['total_sol_vol_100s'], state['unique_wallets_count_100s'], default=0.0)
    state['avg_tx_per_wallet_100s'] = safe_divide(tx_count, state['unique_wallets_count_100s'], default=0.0) # Renamed from churn_ratio

    state['net_sol_vol_100s'] = state['buy_sol_vol_100s'] - state['sell_sol_vol_100s']
    state['net_token_vol_100s'] = state['buy_token_vol_100s'] - state['sell_token_vol_100s']

    state['buy_sell_tx_ratio_100s'] = safe_divide(state['buy_tx_count_100s'], state['sell_tx_count_100s'])
    state['buy_sell_sol_vol_ratio_100s'] = safe_divide(state['buy_sol_vol_100s'], state['sell_sol_vol_100s'])
    state['unique_buyers_ratio_100s'] = safe_divide(state['unique_buyers_count_100s'], state['unique_wallets_count_100s'], default=0.0)
    state['unique_sellers_ratio_100s'] = safe_divide(state['unique_sellers_count_100s'], state['unique_wallets_count_100s'], default=0.0)

    # --- LP State Features ---
    lp_sol_start = state['lp_sol_start_100s']
    lp_sol_end = state['lp_sol_end_100s']
    lp_token_start = state['lp_token_start_100s']
    lp_token_end = state['lp_token_end_100s']
    state['lp_sol_change_100s'] = lp_sol_end - lp_sol_start
    state['lp_token_change_100s'] = lp_token_end - lp_token_start
    state['lp_sol_change_pct_100s'] = safe_divide(state['lp_sol_change_100s'], lp_sol_start)
    state['lp_token_change_pct_100s'] = safe_divide(state['lp_token_change_100s'], lp_token_start)

    # --- Implied Price Features ---
    state['price_start_100s'] = safe_divide(lp_sol_start, lp_token_start)
    state['price_end_100s'] = safe_divide(lp_sol_end, lp_token_end)
    # Ensure prices are valid before calculating change
    if pd.notna(state['price_start_100s']) and pd.notna(state['price_end_100s']):
        state['price_change_100s'] = state['price_end_100s'] - state['price_start_100s']
        state['price_change_pct_100s'] = safe_divide(state['price_change_100s'], state['price_start_100s'])
    else:
        state['price_change_100s'] = np.nan
        state['price_change_pct_100s'] = np.nan
    state['avg_price_per_buy_tx_100s'] = safe_divide(state['buy_sol_vol_100s'], state['buy_token_vol_100s'])
    state['avg_price_per_sell_tx_100s'] = safe_divide(state['sell_sol_vol_100s'], state['sell_token_vol_100s'])

    # --- Timing and Span Features ---
    state['slot_span_100s'] = state['last_slot_100s'] - state['first_slot_100s']
    creation_slot = creation_slots_map.get(mint, state['first_slot_100s']) # Get actual creation slot
    state['first_slot_offset_100s'] = state['first_slot_100s'] - creation_slot
    state['last_slot_offset_100s'] = state['last_slot_100s'] - creation_slot

    if pd.notna(state['first_time_100s']) and pd.notna(state['last_time_100s']) and state['last_time_100s'] > state['first_time_100s']:
        time_diff = state['last_time_100s'] - state['first_time_100s']
        state['time_span_seconds_100s'] = time_diff.total_seconds()
    else:
         state['time_span_seconds_100s'] = 0.0 # Or NaN if preferred

    # --- Rate Features ---
    slot_span_eff = state['slot_span_100s'] + 1 # Use duration + 1 for rate calculation
    time_span_eff = state['time_span_seconds_100s'] if state['time_span_seconds_100s'] > 0 else 1.0 # Avoid zero division

    state['tx_per_slot_100s'] = safe_divide(tx_count, slot_span_eff, default=0.0)
    state['tx_per_second_100s'] = safe_divide(tx_count, time_span_eff, default=0.0)
    state['sol_vol_per_slot_100s'] = safe_divide(state['total_sol_vol_100s'], slot_span_eff, default=0.0)
    state['sol_vol_per_second_100s'] = safe_divide(state['total_sol_vol_100s'], time_span_eff, default=0.0)
    state['lp_sol_rate_per_slot_100s'] = safe_divide(state['lp_sol_change_100s'], slot_span_eff, default=0.0)
    state['lp_sol_rate_per_second_100s'] = safe_divide(state['lp_sol_change_100s'], time_span_eff, default=0.0)
    state['new_wallet_rate_per_slot_100s'] = safe_divide(state['unique_wallets_count_100s'], slot_span_eff, default=0.0)
    state['new_wallet_rate_per_second_100s'] = safe_divide(state['unique_wallets_count_100s'], time_span_eff, default=0.0)


    # --- NEW V2 Features Finalization ---

    # (A) Momentum & Acceleration
    tx_first_half = state['tx_count_first_half']
    tx_second_half = state['tx_count_second_half']
    sol_vol_first_half = state['sol_vol_first_half']
    sol_vol_second_half = state['sol_vol_second_half']
    lp_sol_start = state['lp_sol_start_100s']
    lp_sol_mid = state['lp_sol_at_half_way_mark']
    lp_sol_end = state['lp_sol_end_100s']

    state['sol_vol_ratio_half_100s'] = safe_divide(sol_vol_second_half, sol_vol_first_half)
    state['tx_count_ratio_half_100s'] = safe_divide(tx_second_half, tx_first_half)

    lp_sol_change_first_half = lp_sol_mid - lp_sol_start
    lp_sol_change_second_half = lp_sol_end - lp_sol_mid
    state['lp_sol_change_first_half'] = lp_sol_change_first_half
    state['lp_sol_change_second_half'] = lp_sol_change_second_half
    state['lp_sol_change_ratio_half_100s'] = safe_divide(lp_sol_change_second_half, lp_sol_change_first_half)

    wallets_first_half = state['wallets_first_half_set']
    wallets_all = state['wallets_second_half_set'] # This actually holds all wallets seen
    unique_wallets_first_half_count = len(wallets_first_half)
    # Wallets seen only in the second half
    unique_new_wallets_second_half_count = len(wallets_all - wallets_first_half)
    state['unique_wallets_first_half_count_100s'] = unique_wallets_first_half_count
    state['unique_new_wallets_second_half_count_100s'] = unique_new_wallets_second_half_count
    state['unique_wallets_ratio_half_100s'] = safe_divide(unique_new_wallets_second_half_count, unique_wallets_first_half_count)

    # (B) Efficiency & Sustainability
    state['lp_sol_per_buy_sol_100s'] = safe_divide(state['lp_sol_change_100s'], state['buy_sol_vol_100s'])
    state['net_sol_efficiency_100s'] = safe_divide(state['lp_sol_change_100s'], state['net_sol_vol_100s']) # Can be tricky if net_sol is negative
    state['price_change_per_net_sol_100s'] = safe_divide(state.get('price_change_pct_100s', np.nan), state['net_sol_vol_100s'])
    state['sell_pressure_ratio_100s'] = safe_divide(state['sell_sol_vol_100s'], state['total_sol_vol_100s'], default=0.0)
    state['lp_sol_sustainability_100s'] = safe_divide(lp_sol_end, state['max_lp_sol_100s'], default=1.0) # If max_lp is 0, ratio is NaN, maybe default to 1?

    # (D) Enhanced Fee/Gas
    state['avg_fee_per_sol_100s'] = safe_divide(state['total_fee_100s'], state['total_sol_vol_100s'])
    state['avg_gas_efficiency_100s'] = safe_divide(state['total_gas_consumed_100s'], state['total_provided_gas_limit_100s'])


    # --- Clean up intermediate structures ---
    # Make sure to delete all sets and temporary V2 fields
    del state['unique_wallets_set'], state['unique_buyers_set'], state['unique_sellers_set']
    del state['wallets_first_half_set'], state['wallets_second_half_set']
    # Remove keys no longer needed after calculation
    keys_to_remove = ['lp_sol_at_half_way_mark']
    for key in keys_to_remove:
        if key in state:
            del state[key]

    # Append the finalized dictionary (state) to the list
    final_features_list.append(state)


# --- Create Final DataFrame ---
features_df = pd.DataFrame(final_features_list)

# Final check for infinities just in case safe_divide didn't catch edge cases
# Replace with NaN is generally preferred for CatBoost/most ML libraries
features_df = features_df.replace([np.inf, -np.inf], np.nan)

print(f"\nGenerated features DataFrame shape: {features_df.shape}")
if not features_df.empty:
    print("Generated Features Head (V2):")
    print(features_df.head())
    print("\nGenerated Features Info (V2):")
    # Check for NaNs introduced by new features
    features_df.info(verbose=True, show_counts=True)
    # print("\nNaN counts per column (V2):")
    # print(features_df.isnull().sum().sort_values(ascending=False))

else:
    print("No features were generated.")

# --- Save the V2 features ---
feature_save_path = os.path.join(path, 'engineered_features_100s_v2.parquet') # <<< CHANGED FILENAME
print(f"\nSaving V2 features to {feature_save_path}...")
try:
    # Ensure datetime columns are handled if needed (Parquet usually handles them)
    # features_df.drop(columns=['first_time_100s', 'last_time_100s'], inplace=True, errors='ignore')
    features_df.to_parquet(feature_save_path, index=False)
    print("V2 Features saved successfully.")
except Exception as e:
    print(f"Error saving V2 features: {e}")


Reloading intermediate features_per_mint...
Successfully reloaded 1061444 mints.
Finalizing features...


TypeError: 'module' object is not callable

In [ ]:
features_df.head()

,mint,tx_count_100s,buy_tx_count_100s,sell_tx_count_100s,total_sol_vol_100s,buy_sol_vol_100s,sell_sol_vol_100s,total_token_vol_100s,buy_token_vol_100s,sell_token_vol_100s,...,last_slot_offset_100s,time_span_seconds_100s,tx_per_slot_100s,tx_per_second_100s,sol_vol_per_slot_100s,sol_vol_per_second_100s,lp_sol_rate_per_slot_100s,lp_sol_rate_per_second_100s,new_wallet_rate_per_slot_100s,new_wallet_rate_per_second_100s
0,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,3,2,1,5.656629e+09,3.694123e+09,1.962506e+09,1.767267e+14,1.176405e+14,5.908613e+13,...,53,21.0,0.055556,0.142857,1.047524e+08,2.693633e+08,3.114106e+07,8.007702e+07,0.037037,0.095238
1,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,1,1,0,2.000000e+09,2.000000e+09,0.000000e+00,6.706250e+13,6.706250e+13,0.000000e+00,...,0,0.0,1.000000,1.000000,2.000000e+09,2.000000e+09,0.000000e+00,0.000000e+00,1.000000,1.000000
2,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,13,8,5,7.874963e+09,4.390652e+09,3.484311e+09,2.427691e+14,1.371176e+14,1.056515e+14,...,94,37.0,0.136842,0.351351,8.289435e+07,2.128368e+08,4.277271e+06,1.098218e+07,0.084211,0.216216
3,6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump,1,1,0,4.000000e+09,4.000000e+09,0.000000e+00,1.262353e+14,1.262353e+14,0.000000e+00,...,0,0.0,1.000000,1.000000,4.000000e+09,4.000000e+09,0.000000e+00,0.000000e+00,1.000000,1.000000
4,DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump,3,2,1,2.574964e+09,1.787482e+09,7.874822e+08,8.606154e+13,6.033722e+13,2.572432e+13,...,12,5.0,0.230769,0.600000,1.980742e+08,5.149929e+08,7.692308e-02,2.000000e-01,0.153846,0.400000
